In [ ]:
#Dependencies
# Selenium, webdriver_manager, pandas, openpyxl 

import selenium
import webdriver_manager
import pandas as pd
import datetime
from datetime import timedelta
import calendar
import time
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:

def abrir_webdriver():

    options = Options()
    options.add_argument("start-maximized")
    options.add_argument("--disable-popup-blocking")
    options.add_argument("--disable-notifications")
    options.add_experimental_option('excludeSwitches', ['enable-logging'])

    return webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

def abrirInboxGmail(js):
    if js == "nojs":
        navegador.get("https://mail.google.com/mail/u/0/h/15tdwyr7eeugx/")
    elif js == "js":
        navegador.get("https://mail.google.com/")

def converteData(data):
    data_momento = str(data).split(' ')[0].split('-')
    data_momento = data_momento[2] + '/' + data_momento[1] + '/' + data_momento[0]
    return data_momento #string

def calculaPrazo(data):
    dias_no_mes = calendar.monthrange(data.year, data.month)[1]
    prazo = data + timedelta(dias_no_mes)
    return prazo

def definirResolucao(cargo_servidor):
    if cargo_servidor == "Professor do Magistério Superior":
        return "Resolução nº. 44/2004/CEPE"
    else:
        return "Resolução nº. 08/2007/CUN"

def obterEmail(nome):

    navegador.get("https://contacts.google.com/search/" + nome)

    email = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'a[aria-label="Enviar e-mail"]')))

    return email.text

def limparQuebrasDeLinha(string):
    string = string.replace("\n", "")  # Apagar quebras de linha
    return string

In [ ]:
#Abrir webdriver e definir wait

navegador = abrir_webdriver()

wait = WebDriverWait(navegador, 20)
    
abrirInboxGmail("js")

input("Faça login no Gmail, volte aqui e aperte enter")

In [ ]:
#Ler tabela e enviar e-mails

tabela = pd.read_excel("tabela_notificacao_ep.xlsx")


for i, nome_servidor in enumerate(tabela["nome"]):

    momento = str(tabela.loc[i, "momento"])
    siape_servidor = str(tabela.loc[i, "siape"])
    cargo_servidor = tabela.loc[i, "cargo"]
    data_momento = converteData(tabela.loc[i, "data_momento"])
    data_fim_prazo = converteData(calculaPrazo(tabela.loc[i, "data_momento"]))
    setor_exercicio = tabela.loc[i, "setor_exercicio"]
    nome_chefia = tabela.loc[i, "nome_chefia"]
    resolucao = definirResolucao(cargo_servidor)

    with open("modelo.txt", "r", encoding="UTF-8") as file:
        modelo = file.read()

    assunto = limparQuebrasDeLinha(modelo.split("##ASSUNTO")[1].split("##FIM_ASSUNTO")[0])
    mensagem = modelo.split("##MENSAGEM")[1].split("##FIM_MENSAGEM")[0]

    mensagem = mensagem.replace("{{momento}}", momento)
    mensagem = mensagem.replace("{{nome_servidor}}", nome_servidor)
    mensagem = mensagem.replace("{{siape_servidor}}", siape_servidor)
    mensagem = mensagem.replace("{{cargo_servidor}}", cargo_servidor)
    mensagem = mensagem.replace("{{setor_exercicio}}", setor_exercicio)
    mensagem = mensagem.replace("{{data_fim_prazo}}", data_fim_prazo)
    mensagem = mensagem.replace("{{resolucao}}", resolucao)

    ##Busca e-mail servidor e chefia

    email_chefia = obterEmail(nome_chefia)

    email_servidor = obterEmail(nome_servidor)

    email_ddcc = "sdcc.ddp.progep@ufes.br"

    abrirInboxGmail("nojs")

    btnEscrever = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/table[2]/tbody/tr/td[1]/table[1]/tbody/tr[1]/td/b/a')))

    btnEscrever.click()

    fldPara = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="to"]')))

    fldCc = wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="cc"]')))

    fldAssunto = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/table[2]/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/form/table[2]/tbody/tr[4]/td[2]/input')))

    fldMsg = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/table[2]/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/form/table[2]/tbody/tr[8]/td[2]/textarea')))

    fldPara.send_keys(email_chefia)

    stringEmailsCC = email_servidor + "; " + email_ddcc;

    fldCc.send_keys(stringEmailsCC)

    fldAssunto.send_keys(assunto)

    fldMsg.send_keys(Keys.CONTROL + "a" + Keys.DELETE)

    fldMsg.send_keys(mensagem)

    btnEnviar = wait.until(EC.visibility_of_element_located((By.XPATH, '/html/body/table[2]/tbody/tr/td[2]/table[1]/tbody/tr/td[2]/form/table[3]/tbody/tr/td/input[1]')))

    btnEnviar.click()

    resultado = navegador.find_element(By.XPATH, "/html/body/table[1]/tbody/tr[3]/td/table/tbody/tr/td")

    print(nome_servidor, " - ", resultado.text)
    